# Java Streams: Functional programming in Java 

In this section we will look at functional programming in Java. We will look at the following topics:
- Functional interfaces
- Lambda expressions
- Method references
- Streams
- Optional
- Collectors


## Functional interfaces

Functional interfaces are interfaces with only **one abstract method**. They are used to define lambda expressions. Java has a lot of predefined functional interfaces like `Predicate`, `Consumer`, `Supplier`, `Function` etc. Let's look at some examples: 


In [ ]:
@FunctionalInterface              //Optional to add this annotation
public interface Predicate<T> {

    /**
     * Evaluates this predicate on the given argument.
     *
     * @param t the input argument
     * @return {@code true} if the input argument matches the predicate,
     * otherwise {@code false}
     */
    boolean test(T t);
}

In [1]:
import java.util.function.*;

new Predicate<String>() {
    @Override
    public boolean test(String s) {
        return s.startsWith("Hello");
    }
}.test("Hello World");


true

## Lambda expressions

Lambda expressions are anonymous functions. They are used to define functional interfaces. Let's look at some examples:

In [2]:
import java.util.function.*;

Predicate<String> predicate = s -> s.startsWith("Hello");
predicate.test("Hello World");
//predicate.test("World");


true

In [3]:
interface MyInterface {
    void doSomething(String s);
}

MyInterface myInterface = s -> System.out.println(s);
myInterface.doSomething("Hello World");

Hello World


In [4]:
Predicate<Integer> predicate = i -> i > 10;
predicate.test(11);

true

In [5]:
Predicate<Integer> predicate1 = i -> i > 10;
Predicate<Integer> predicate2 = i -> i < 20;
predicate1.and(predicate2).test(15);


true

In [7]:
Predicate<Integer> predicate1 = i -> i > 10;
Predicate<Integer> predicate2 = i -> i < 20;
predicate1.negate().or(predicate2).negate().test(9);

predicate1.negate().or(predicate2.negate()).test(9);

CompilationException: 

### A lambda expression can have a body


In [10]:
Predicate<Integer> predicate = i -> {
    System.out.println("Hello World");
    return i > 10;
};
predicate.test(11);

Hello World


true

#### Passing lambda expressions as arguments

In [32]:
import java.util.function.*;

static void printIf(Predicate<String> predicate, String str) {
    if (predicate.test(str)) {
        System.out.println(str);
    }
}

Predicate<String> one = new Predicate<String>() {
    @Override
    public boolean test(String s) {
        return s.startsWith("Hello");
    }
};
Predicate<String> two = s -> s.startsWith("Hello");

printIf(one, "Hello World");
printIf(two, "Hello World");
printIf(s -> s.startsWith("Hello"), "Hello World");

Hello World
Hello World
Hello World


## Important 

Behind the syntactic sugar of lambda expressions, Java still wraps these into functional interfaces. So, Java treats a lambda expression as an Object, which is the true first-class citizen in Java.

TIP: IDE will automatically help you when you write `new Predicate()...` and offers help to rewrite it to a lambda.

### Different functional interfaces

<img src="https://i.imgur.com/rtjKUji.png">

#### Consumer

A Consumer is a functional interface that takes an argument and returns nothing. It is used to perform some operations on the argument. Let's look at some examples:


In [11]:
Consumer<String> consumer = s -> System.out.println(s);
consumer.accept("Hello World");

Hello World


In [13]:
Consumer<String> consumer = s -> System.out.println(s);
Consumer<String> consumer2 = s -> System.out.println(s.toUpperCase());
consumer.andThen(consumer2).accept("Hello World");

Hello World
HELLO WORLD


In [14]:
List<String> list = List.of("Hello", "World");

static void loopList(List<String> list, Consumer<String> consumer) {
    for (String s : list) {
        consumer.accept(s);
    }
}
loopList(list, s -> s.upperCase());
loopList(list, s -> System.out.println(s));
loopList(list, s -> System.out.println(s.toUpperCase()));
loopList(list, s -> System.out.println(s.length()));

Hello
World
HELLO
WORLD
5
5


### Supplier

A Supplier is a functional interface that takes no arguments and returns a value. It is used to supply values. Let's look at some examples:

In [16]:
Supplier<String> supplier = () -> "Hello World";

String s = supplier.get();
s;

Hello World

Supplier can be used for lazy evaluation like this:

In [ ]:
Supplier<String> supplier = () -> {
    System.out.println("Hello World");
    return "Hello World";
};

### Caching supplier

Let's code an example for a caching supplier, and make sure that the supplier is only called once:

In [17]:
import java.util.function.*;

class LazySupplier<T> implements Supplier<T> {

    private final Supplier<T> supplier;
    private T value;

    public LazySupplier(Supplier<T> supplier) {
        this.supplier = supplier;
    }

    @Override
    public T get() {
        if (value == null) {
            //computation which takes 5 minutes
            value = supplier.get();
        }
        return value;
    }
}

var lazy = new LazySupplier<>(() -> {
    System.out.println("Calling Supplier");
    return "Hello World";
});
System.out.println(lazy.get());
lazy.get();


Calling Supplier
Hello World


Hello World

### Function

A `Function` is a functional interface that takes an argument and returns a value. It is used to transform values. The signature of the function is `T -> R`.

Let's look at some examples:

In [8]:
import java.util.function.*;

Function<String, String> identity = s -> s
Function<String, String> function = s -> s.toUpperCase();
function.apply("Hello World");


HELLO WORLD

### Chaining functions


In [18]:
Function<String, String> toUpperCase = s -> {
  System.out.println("toUpperCase");
  return s.toUpperCase();
};
Function<String, String> removeQuotes = s -> {
  System.out.println("removeQuotes");
  return s.replaceAll("\"", "");
};

System.out.println("\"Hello World\"");
toUpperCase.andThen(removeQuotes).apply("\"Hello World\"");
toUpperCase.compose(removeQuotes).apply("\"Hello World\"");

"Hello World"
toUpperCase
removeQuotes
removeQuotes
toUpperCase


HELLO WORLD

### BiFunction

A `BiFunction` is a functional interface that takes two arguments and returns a value. It is used to transform values. The signature of the function is `(T, U) -> R`.
Let's look at some examples:

In [13]:
import java.util.function.*;

BiFunction<String, String, String> concatenate = (s1, s2) -> s1 + s2;
concatenate.apply("Hello", "World");

HelloWorld

## Streams

Stream in Java give a functional approach to process collections. A sequence of objects, known as a stream, provides support for a variety of methods that can be sequentially applied to achieve the intended outcome.



<img src="https://media.geeksforgeeks.org/wp-content/uploads/20210706120537/JavaStream.png"/>

- Stream takes the input from a collection (does not hold any elements itself)
- Stream does not manipulate the collection itself
- Each operation is done by lazy evaluation (remember lambdas) and returns a new stream

In [24]:
import java.util.*;
import java.util.stream.*;

List<Integer> numbers = List.of(1, 2, 3, 4, 5, 6, 7, 8, 9);

//List<Integer> evenNumbers = 
    
    numbers.stream()
    .filter(i -> i % 2 == 0)
    .map(i -> i * 2)
    .count();
//evenNumbers;



4

## Intermediate operations

These are the operations that return a `Stream`, for example: `filter`, `map`, `flatMap`, `sorted`. And you can combine those:


In [28]:
record Cat(String name, int age) {} 

List<Cat> cats = List.of(new Cat("Felix", 5), new Cat("Tom", 12));

//Let's say we want to find all the cats above the age of 10 and return their names
Predicate<Cat> above10 = cat -> cat.age() >= 10;
Function<Cat, String> name = cat -> cat.name();
Stream<Cat>    filteredAgeStream = cats.stream().filter(above10);         
Stream<String> onlyNameStream    = filteredAgeStream.map(name);


onlyNameStream.toList();  //short for collect(Collectors.toList())

//In one go:
cats.forEach(cat -> System.out.println(cat));
cats.stream()
    .filter(cat -> cat.age() >= 10)
    .map(cat -> cat.name())
    .toList(); 
    
// Try to imagine to do this with for loops :-)

Cat[name=Felix, age=5]
Cat[name=Tom, age=12]


[Tom]

The `filter` function takes a `Predicate` as a lambda and is implemented as follows in one of the `Stream` subclasses:

```java
    @Override
    public final Stream<P_OUT> filter(Predicate<? super P_OUT> predicate) {
        Objects.requireNonNull(predicate);
        return new StatelessOp<P_OUT, P_OUT>(this, StreamShape.REFERENCE,
                                     StreamOpFlag.NOT_SIZED) {
            @Override
            Sink<P_OUT> opWrapSink(int flags, Sink<P_OUT> sink) {
                return new Sink.ChainedReference<P_OUT, P_OUT>(sink) {
                    @Override
                    public void begin(long size) {
                        downstream.begin(-1);
                    }

                    @Override
                    public void accept(P_OUT u) {
                        if (predicate.test(u))
                            downstream.accept(u);
                        
                    }
                };
            }
        };
    }
```

Same as for `map` which takes a function which takes a `Cat` and returns possible another type.

In [33]:
record Cat(String name, int age) {} 

List<Cat> cats = List.of(new Cat("Felix", 5), new Cat("Tom", 12));
cats.stream().map(cat -> cat).toList();                             //returning itself is fine, called identity function.

[Cat[name=Felix, age=5], Cat[name=Tom, age=12]]

### Flat map 

The `flatMap` function is similar to the map function, but with an important difference. While `map` transforms each element of the stream into another single element, `flatMap` transforms each element into a stream of elements and then flattens the resulting streams into a single stream.

- **Mapping Function:**
  - `flatMap` takes a mapping function as an argument.
  - This mapping function should return a stream for each input element.

- **Flattening:**
  - The output streams from the mapping function are then flattened into a single stream.
  - This means that the elements of the individual streams are concatenated into one flat stream.
    
In summary, `flatMap` is a powerful operation that is useful when dealing with nested structures. It allows you to apply a transformation to each element of a stream, where the transformation results in a stream of values, and then flattens these streams into a single stream of values.

In [29]:
import java.util.*;

//<R> Stream<R> flatMap(Function<? super T, ? extends Stream<? extends R>> mapper)


List<List<Integer>> listOfLists = List.of(
  List.of(1, 2, 3),
  List.of(4, 5, 6),
  List.of(7, 8, 9)
);

Stream<List<Integer>> streamOfList = listOfLists.stream();
Stream<Integer> streamOfInts = streamOfList.flatMap(l -> l.stream());
streamOfInts.toList();

CompilationException: 

In [53]:
List<String> words = List.of("Cats", "are", "the", "best");

words.stream().flatMap(str -> Stream.of(str.substring(0, 2))).toList();


//which you can also write as
words.stream().map(str -> str.substring(0,2)).toList();
words.stream().map(str -> Stream.of(str.substring(0,2))).flatMap(s -> s).toList();


[Ca, ar, th, be]

## Collectors

Collectors provide a convenient way to transform the elements of a Stream into a different form, such as a List, Set, Map, or even a custom container. They encapsulate the logic for mutable reduction, making it easier to perform operations like grouping, partitioning, or summarizing data.

In [56]:
import java.util.stream.*;

Stream.of("Cats").collect(Collectors.toMap(s -> s.length(), s -> s));

{4=Cats}

In [30]:
var words = List.of("Cats", "are", "the", "best");
words.stream().collect(Collectors.joining(", "));

Cats, are, the, best

In [32]:
import java.util.stream.*;

var numbers = List.of(1, 2, 3, 4);
numbers.stream().collect(Collectors.groupingBy(number -> number % 2 == 0));

// var words = List.of("Cats", "are", "the", "Cats");
// words.stream().collect(Collectors.groupingBy(s -> s.length()));
// words.stream().collect(Collectors.groupingBy(s -> s));


{false=[1, 3], true=[2, 4]}

In [9]:
var words = List.of("Cats", "are", "the", "Dogs");

words.stream().collect(Collectors.partitioningBy(s -> s.length() > 3));


{false=[are, the], true=[Cats, Dogs]}

## Short circuit operations

Short-circuiting operations allow computations on infinite streams to complete in finite time. They are called short-circuiting operations because they may not have to process the entire stream to produce a result. For example, `limit(n)` doesn’t need to read the entire stream to produce a result with n elements. Similarly, `findFirst()` does not need to examine all the stream elements to find one.

In [33]:
import java.util.stream.*;
import java.util.*;

Stream<Integer> infiniteStream = Stream.iterate(0, i -> i + 1);
infiniteStream.limit(20).collect(Collectors.toList());


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [35]:
import java.util.stream.*;
import java.util.*; 

Stream.of("Cats", "are", "the", "best").anyMatch(s -> s.startsWith("c"));

false

In [22]:
import java.util.stream.*;
import java.util.*; 

Stream.of("Cats", "are", "the", "best").filter(s -> s.startsWith("C")).findFirst();

Optional[Cats]

## Optional

The previous example leads us to the `Optional` type. Optional is a container object that may or may not contain a non-null value. Optional is not a functional interface, instead it is a final class with some useful methods. Let's look at some examples:

In [24]:
import java.util.*;


Optional<String> optional = Optional.of("Hello World"); 
System.out.println(optional.get());
System.out.println(optional.isPresent());
System.out.println(optional.isEmpty());

Hello World
true
false


In [25]:
import java.util.*;

Optional<String> optional = Optional.empty();
System.out.println(optional.isPresent());
System.out.println(optional.isEmpty());

false
true


Optionals can be used to avoid null pointer exceptions. So instead of writing:

```java
public Optional<String> name(String optionalName) {
  return null;
}

//you can write
public Optional<String> name() {
  return Optional.empty();
}
```

Option

For the caller it is now clear the method `name()` can return an empty value and the caller can decide what to do with it.


In [38]:
import java.util.*;

Optional<String> optional = Optional.of("Hello World");
optional.ifPresent(s -> System.out.println(s));
Optional<String> s = optional.map(s -> s.toUpperCase()); 
s;
s.ifPresent(s -> System.out.println(s));

Hello World
HELLO WORLD


## Optional flatMap

Optional also has a `flatMap` method which is similar to the `flatMap` method on streams. Let's look at some examples:

In [43]:
import java.util.*;

record Person(String name, Optional<Address> address) {}
record Address(String street, Optional<String> city) {}

Person person = new Person("John", Optional.of(new Address("Prinsengracht 23", Optional.of("Amsterdam"))));
System.out.println(person.address().map(a -> a.city()));

//With flatMap
person.address().flatMap(a -> a.city()).orElse("No city");

Optional[Optional[Amsterdam]]


Amsterdam

### Optional with streams

One of the most common use cases for Optional is to use it with streams. Let's look at some examples:

In [ ]:
import java.util.*;

List<String> words = List.of("Cats", "are", "the", "best");
words.

In [2]:
boolean bool = true;              // boolean is a primitive type, range is true or false
byte b = -15;                     // byte is a primitive type, range is -128 to 127
short s = 1;                      // short is a primitive type, range is -32768 to 32767
int i = 1;                        // int is a primitive type, range is
long l = 1L;                      // long is a primitive type, min value is -2^63, max value is 2^63-1
float f = 1.0f;                   // float is a 32-bit floating-point number, min value is 1.4e-045, max value is 3.4e+038
double pi = 3.1416;               // double is 64-bit floating-point number, min value is 4.9e-324, max value is 1.8e+308
char letterOmega = 'Ω';           // 16-bit Unicode character
String str = "Hello, world! 😊";
int million = 1_000_000;

System.out.println("bool = " + bool);
System.out.println("b = " + b);
System.out.println("i = " + i);
System.out.println("l = " + l);
System.out.println("f = " + f);
System.out.println("d = " + pi);
System.out.println("c = " + letterOmega);
System.out.println("s = " + s);
System.out.println("str = " + str);

bool = true
b = -15
i = 1
l = 1
f = 1.0
d = 3.1416
c = Ω
s = 1
str = Hello, world! 😊


### Conversion

We can also convert types from one to another like:

<img src="https://static.javatpoint.com/core/images/type-casting-in-java.png" />

In [3]:
byte a = 2;
byte b = 3;
byte c = 1;

int d = a * b + c;
System.out.println("d = " + d);

long l = 30L;
//int t = l; //Not without an explicit cast

d = 7


In [4]:
float f = 1.2f;
double __ = 10 * 2.5;
f = (float) (Math.PI * 10);

f;

31.415926

### Java Types 

These are the reference types a programmer can define and invoke methods on. These objects are always created on the heap. Let's look at some examples:

#### Arrays

In [4]:
int array[][] = {{1, 2, 3}, {4, 5, 6}};
System.out.println(array.length);
for (int i = 0; i < 2; i++) {
  for (int j = 0; j < 3; j++) {
    System.out.print(array[i][j] + " ");
  }
  System.out.println();
}



2
1 2 3 
4 5 6 


#### Strings

In Java a String is kind of a special type as you can define it as follows:

In [8]:
String hello = "Hello World!";

String longHello = """
    Hello World!
    From the Java training
    """;
hello;
longHello;

Hello World!
From the Java training


#### Null 

Null is a literal, to which you point if there is no value. 



In [10]:
String s = null;
//System.out.println(s.length()); //NullPointerException

//What will this print?
s = s + "Hello";
System.out.println("s = " + s);

String.valueOf((Object)null);

s = nullHello


null

#### Simple type

Let's define our own type

In [11]:
class Person { 
    String name = "John";
    int age = 32;
}

Person person = new Person();
person.name;

John

Every type has implictly `java.lang.Object` as parent.

In [1]:
String str = "Test";

str instanceof Object

true

In [ ]:
person instanceof Person

In [ ]:
person instanceof Object

#### Wrapper classes

Each Java primitive type has a wrapper class and Java support autoboxing and unboxing between the two

In [15]:
boolean primitive = true;

Boolean wrapper = primitive;
wrapper;

true

In [16]:
Boolean wrapper = true;
boolean primitive = wrapper;

primitive;

true

Primitives don't have concept of `null` so

In [18]:
Boolean wrapper = null;
boolean primitive = wrapper; //crashes at runtime

EvalException: Cannot invoke "java.lang.Boolean.booleanValue()" because "REPL.$JShell$45C.wrapper" is null

# Operators

On primitive types Java defines operators. On types we can define our own methods.

https://www.startertutorials.com/corejava/wp-content/uploads/2014/10/Operators.jpg

<img src="https://www.startertutorials.com/corejava/wp-content/uploads/2014/10/Operators.jpg"/>

In [ ]:
int i = 10;

System.out.println("i_1 = " + i++);
System.out.println("i_1 = " + i);

System.out.println("i_2 = " + ++i);
System.out.println("i_2 = " + i);

boolean a = true;
boolean b = false;

System.out.println(a && b);
System.out.println(a || b);
System.out.println(!a);
System.out.println(a ^ !b);
System.out.println(a ^ b);

String a = "a";
String b = "b";
a + b

## Control flow

In this section we will look some of language control statements.

### If statement

In [ ]:
 String str = "Hello World";

if (str.startsWith("hello")) {
  System.out.println("Starts with hello");
} else if (str.startsWith("Hello")) {
  System.out.println("Starts with Hello");
} else {
  System.out.println("No match found");
}

<img src="https://scaler.com/topics/images/nesting-ternary-operator.webp"/>

In [ ]:
String str = "Hello World";
String result = str.startsWith("hello") ? "Starts with hello" : "No match found";
System.out.println(result);

<img src="https://static.javatpoint.com/images/java-loops.png"/>

### For loops

In [ ]:
String[] words = {"Hello", "World"};

for (int i = 0; i < words.length; i++) {
  System.out.println("#" + i  + " is " + words[i]);
}

In [ ]:
String[] words = {"Hello", "World"};

//for each construct 
for (String word : words) {
  System.out.println(word);
}

### While loops

In [7]:
String[] words = {"Hello", "World"};

int i = 0;
while (i < words.length) {
  System.out.println("#" + i  + " is " + words[i]);
  i = i + 1;
}

#0 is Hello
#1 is World


In [8]:
String[] words = {"Hello", "World"};

//for loop as while --> not recommended
int k = 0;
for (; k < words.length; k++ ) {
  System.out.println("#" + k + " is " + words[k]);
}


#0 is Hello
#1 is World


In [9]:
//do while loop
String[] words = {"Hello", "World"};

int l = 0;
do {
  System.out.println("#" + l + " is " + words[l]);
  l++;
} while (l < words.length);

#0 is Hello
#1 is World


### Switch statements

In [10]:
//Traditional switch statement 

String str = "Hello World";

switch (str) {
  case "Hello World":
    System.out.println("Hello World");
    break;
 case "Hello":
    System.out.println("Hello");
    break;
 default:
    System.out.println("No match found");
}

Hello World


In [ ]:
//more power
String str = "Hello World";

System.out.println(switch (str) {
  case "Hello World" -> "Hello World";
  case "Hello" -> "Hello";
  default -> "No match found";     //default is mandatory
});

In [ ]:
//more power
//Object str = "Hello World";
Object str = 1;

System.out.println(switch (str) {
  case String s -> s;
  case Integer i -> i;
  default -> "No match found";     //default is mandatory
});

In [2]:
Object o = null;
//Object o = "Hello World";
//Object o = "";
//String o  = "H"

System.out.println(switch (o) {
  case null -> "Null";
  case String s when s.isBlank() -> "Blank";
  case String s -> s;
  case Integer i -> i;
  default -> "No match found";     //default is mandatory
});

CompilationException: 

In [ ]:
// or write it with if statements like this

if (o == null) {
  System.out.println("Null");
} else if (o instanceof String s && s.isBlank()) {
  System.out.println("Blank");
} else if (o instanceof String s) {
  System.out.println(s);
} else if (o instanceof Integer i) {
  System.out.println(i);
} else {
  System.out.println("No match found");
}

### Continue / break

<img src="https://static.javatpoint.com/core/images/branching-statements-in-java.png"/>

In [ ]:
String[] words = {"Hello", "World"};
for (int i = 0; i < words.length; i++) {
  if (i == 0) {
    System.out.println("Position %d contains word %s".formatted(i, words[i]));
    break;
  }
}

In [ ]:
outerMost:
  for (int j = 0; j < 5; j++) {

  innerMost:
    for (int k = 1; k < 3; k++) {
      System.out.println("j = " + j + " and k = " + k);

      // Terminating the outerMost loop
      if (j == 3)
        break outerMost;
      }
    }

## Logical operators

In [ ]:
int i = 10;

System.out.println("i_1 = " + i++);
System.out.println("i_1 = " + i);


In [ ]:
int j = 10;

System.out.println("i_2 = " + ++j);
System.out.println("i_2 = " + j);

In [3]:
boolean a = true;
boolean b = false;

System.out.println(a && b);
System.out.println(a || b);
System.out.println(!a);
System.out.println(a ^ !b);
System.out.println(a ^ b);

false
true
false
false
true


# Assignment 1

- Sum
- Triangle
- Palindrome